In [1]:
import pandas as pd

train_normal = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_normal.csv')
train_one = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_1.csv')
train_two = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_2.csv')
train_three = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_3.csv')
train_four = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_4.csv')
train_five = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_5.csv')
train_six = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_6.csv')
train_seven = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_7.csv')
train_eight = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_8.csv')

test_normal = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_normal.csv')
test_one = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_1.csv')
test_two = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_2.csv')
test_three = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_3.csv')
test_four = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_4.csv')
test_five = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_5.csv')
test_six = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_6.csv')
test_seven = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_7.csv')
test_eight = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_8.csv')

In [ ]:
train_normal['label']=0
train_one['label']=1
train_two['label']=2
train_three['label']=3
train_four['label']=4
train_five['label']=5
train_six['label']=6
train_seven['label']=7
train_eight['label']=8

test_normal['label']=0
test_one['label']=1
test_two['label']=2
test_three['label']=3
test_four['label']=4
test_five['label']=5
test_six['label']=6
test_seven['label']=7
test_eight['label']=8


In [ ]:
train_merge = pd.concat([train_normal, train_one, train_two, train_three, train_four, train_five, train_six, train_seven, train_eight], ignore_index=True)
test_merge = pd.concat([test_normal, test_one, test_two, test_three, test_four, test_five, test_six, test_seven, test_eight], ignore_index=True)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, :-1].values  # 마지막 열을 제외한 데이터
        self.labels = dataframe.iloc[:, -1].values  # 마지막 열을 라벨 데이터로 사용

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': torch.tensor(self.data[idx], dtype=torch.float32),
                  'label': torch.tensor(self.labels[idx], dtype=torch.long)}
        return sample

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(train_merge.iloc[:, :-1].values)
X_test_normalized = scaler.transform(test_merge.iloc[:, :-1].values)

n_components_pca = 12  # 주성분 개수
pca = PCA(n_components=n_components_pca)

X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

train_merge_pca = pd.DataFrame(data=np.hstack((X_train_pca, train_merge['label'].values.reshape(-1, 1))),
                               columns=[f'pca_{i}' for i in range(n_components_pca)] + ['label'])
test_merge_pca = pd.DataFrame(data=np.hstack((X_test_pca, test_merge['label'].values.reshape(-1, 1))),
                              columns=[f'pca_{i}' for i in range(n_components_pca)] + ['label'])

In [ ]:
train_dataset_pca = CustomDataset(train_merge_pca)
train_loader_pca = DataLoader(train_dataset_pca, batch_size=64, shuffle=True, drop_last=True)

test_dataset_pca = CustomDataset(test_merge_pca)
test_loader_pca = DataLoader(test_dataset_pca, batch_size=64, shuffle=False, drop_last=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from resnet_1d import *

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

def create_models(num_classes=9):
    model_18 = resnet18(num_classes=9, pretrained=False)
    model_34 = resnet34(num_classes=9, pretrained=False)
    
    return model_18,model_34

model_18, model_34 = create_models()

In [ ]:
import os 
model = model_18

# resnet18
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_18.parameters(), lr=2e-4)

## train 및 valid
os.makedirs('./result', exist_ok=True)
from sklearn.metrics import f1_score, classification_report
import numpy as np
from tqdm.notebook import trange
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader_pca)
valid_loss = np.Inf  # train acc
f1_ = 0  # val f1
epoch_in = trange(100, desc='training')
best_acc=0



for epoch in epoch_in:
    model.to(device)
    model.train()
    running_loss = 0.0
    correct = 0
    total=0
    
    preds_ = []
    targets_ = []

    for batch_idx, train_dict in enumerate(train_loader_pca):

        inputs = train_dict['data'].to(device).float()
        inputs = inputs.reshape(64,1,1,12)
        labels = train_dict['label'].to(device).long()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)
        # if (batch_idx) % 1000 == 0:
        #     print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        #            .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')

    
    batch_loss = 0
    total_t=0
    correct_t=0    
    
    with torch.no_grad():
        model.to(device)
        model.eval()
        for test_dict in (test_loader_pca):
            data_t = test_dict['data'].to(device).float()
            data_t = data_t.reshape(64,1,1,12)
            #print(data_t)
            
            target_t =  test_dict['label'].to(device).long()
            #print(target_t)
            
            outputs_t = model(data_t)
            
            ####################### f1 score ################################
            pred = outputs_t.argmax(dim=1).to(device)
            target = target_t.view_as(pred).to(device)

            preds_.append(pred)
            targets_.append(target)
            
            # f1_score += f1(pred, target)
            ##################################################################
            
            
            
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)

        #print(total_t)

        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(test_loader_pca))
        network_learned = batch_loss < valid_loss_min

        preds_ = torch.cat(preds_).detach().cpu().numpy()
        targets_ = torch.cat(targets_).detach().cpu().numpy()

        f1score = f1_score(targets_,preds_,  average='macro')
        if best_acc < f1score:
            best_acc = f1score
            with open("./result/iot20d_pca+mfcc_res18_0112.txt", "a") as text_file:
                print('epoch=====',epoch, file=text_file)
                print(classification_report(targets_, preds_, digits=4), file=text_file)
            torch.save(model, f'./result/iot20d_pca+mfcc_res18_0112.pt') 
        epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_acc}")